### Generating Hamiltonians for exact results

In this tutorial I'll be showing how to build and obtain intersting information from hamiltonains using the QCOM package.

### Rydberg Hamiltonian
The hamiltonain we would like to build is the following:

\begin{equation}
H = \frac{\Omega}{2}\sum_i(\ket{g_i}\bra{r_i} + \ket{r_i}\bra{g_i})-\Delta\sum_i  n_i +\sum_{i<j}V_{ij}n_in_j,
\end{equation}

This assume $\hbar$ units. The inputs to our function will be the number of atoms, $\Omega$, $\Delta$, $\frac{R_b}{\alpha}$ and a flag which tells the solver whether or not to use Periodic Boundary Conditions. As always we will have the additionally show_progress flag that determines whether or not you get progress updates. By default this flag is set to false, so you must set it to true if you want to see progress updates.

### 1-D Rydberg Chain

The code below builds the hamiltonian for a 1D chain of rydberg atoms. See the figure below for the exact geometry being built. The numbers within the atoms correspond to the indexing convention

<img src = "../images/11_atom_Chain_Structure.png" style = "width:50%">

In [1]:
# import qcom module

import qcom as qc
import numpy as np

In [2]:
import numpy as np

# Parameters for the Rydberg Hamiltonian
num_atoms = 6  # Number of atoms in the chain
Omega = 5 * np.pi  # Rabi frequency in MHz
Delta = 3.5 * Omega  # Detuning in MHz
a = 3.56  # Lattice spacing in μm
pbc = True  # Periodic boundary conditions
show_progress = True  # Print progress of the Hamiltonian construction

# Construct the Hamiltonian for the Rydberg model on a chain

hamiltonian = qc.build_rydberg_hamiltonian_chain(num_atoms, Omega, Delta, a, pbc, show_progress)
print(hamiltonian.toarray())

Starting: Building Rydberg Hamiltonian (Chain)...
Task: Building Rydberg Hamiltonian (Chain) | Progress: 100.00% | Elapsed: 0.27s | Remaining: 0.00s
Completed: Building Rydberg Hamiltonian (Chain). Elapsed time: 0.27 seconds.
[[ 0.00000000e+00  7.85398163e+00  7.85398163e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.85398163e+00 -5.49778714e+01  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.85398163e+00  0.00000000e+00 -5.49778714e+01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.05606547e+04
   0.00000000e+00  7.85398163e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   1.05606547e+04  7.85398163e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  7.85398163e+00
   7.85398163e+00  1.59234488e+04]]


Additionally if you have $\frac{R_b}{\alpha}$ and not $\alpha$ by itself you can obtain it through the following relationship. C6 is a constant and can always be used in this context

In [3]:
# Parameters for the Rydberg Hamiltonian
num_atoms = 6  # Number of atoms in the chain
Omega = 5 * np.pi  # Rabi frequency in MHz
Delta = 3.5 * Omega  # Detuning in MHz
C6 = 5420503  # Van der Waals interaction strength in MHz * μm^6
Rba = 2.35 # Rydberg blockade radius divided by the lattice spacing
a = ((C6 / Omega) ** (1 / 6)) / Rba  # Lattice spacing in μm
pbc = True  # Periodic boundary conditions

# Construct the Hamiltonian for the Rydberg model on a chain

hamiltonian = qc.build_rydberg_hamiltonian_chain(num_atoms, Omega, Delta, a, pbc)
print(hamiltonian.toarray())

state = 0

eigen_value, eigenstate = qc.find_eigenstate(hamiltonian, state)

print(f"Eigenvalue: {eigen_value}")
print(f"Eigenstate: {eigenstate}")

[[ 0.00000000e+00  7.85398163e+00  7.85398163e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.85398163e+00 -5.49778714e+01  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.85398163e+00  0.00000000e+00 -5.49778714e+01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.04907089e+04
   0.00000000e+00  7.85398163e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   1.04907089e+04  7.85398163e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  7.85398163e+00
   7.85398163e+00  1.58185302e+04]]
Eigenvalue: -105.87152240165011
Eigenstate: [ 4.80155283e-02 -1.07874734e-01 -1.07874734e-01  6.52539746e-04
 -1.07874734e-01  4.90513046e-02  6.52539746e-04 -7.50402870e-05
 -1.07874734e-01  5.51602999e-01  4.90513046e-02 -1.79175606e-03
  6.52539746e-04 -1.79175606e-03 -7.50402870e-05  3.70513993e-06
 -1.07874734e-01  4.90513046e-02  5.51602999e-01 -1.79175606

### Rybderg Ladder 
Moving to 2 dimensions we can generate the ladder structure shown in the image below. The key input here is the factor $\rho$ which determines the ratio between x and y distances. In the image below $\rho$ = 2 meaning the y spacing is 2 times that of the x spacing. Conversely if $\rho$ = 0.5 the y spacing would be half that of the x spacing. Once again the numbers within the atoms correspond to their indexing convention

<img src = "../images/Ladder_Structure.png" style = "width:50%">

In [2]:
# Parameters for the Rydberg Hamiltonian
num_atoms = 6  # Number of atoms in the ladder
Omega = 5 * np.pi  # Rabi frequency in MHz
Delta = 3.5 * Omega  # Detuning in MHz
C6 = 5420503  # Van der Waals interaction strength in MHz * μm^6
Rba = 2.35
rho = 2.0  # Ratio of the rung spacing to the lattice spacing
a = ((C6 / Omega) ** (1 / 6)) / Rba  # Lattice spacing in μm
pbc = True  # Periodic boundary conditions
show_progress = True  # Print progress of the Hamiltonian construction

state = 0

# Construct the Hamiltonian for the Rydberg model on a ladder
hamiltonian = qc.build_rydberg_hamiltonian_ladder(num_atoms, Omega, Delta, a, rho, pbc, show_progress)
print(hamiltonian.toarray())

eigen_value, eigenstate = qc.find_eigenstate(hamiltonian, state)
print(f"Eigenvalue: {eigen_value}")
print(f"Eigenstate: {eigenstate}")

Starting: Building Rydberg Hamiltonian (Ladder)...
Task: Building Rydberg Hamiltonian (Ladder) | Progress: 100.00% | Elapsed: 0.37s | Remaining: 0.00s
Completed: Building Rydberg Hamiltonian (Ladder). Elapsed time: 0.37 seconds.
[[ 0.00000000e+00  7.85398163e+00  7.85398163e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.85398163e+00 -5.49778714e+01  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.85398163e+00  0.00000000e+00 -5.49778714e+01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.06609358e+04
   0.00000000e+00  7.85398163e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   1.06609358e+04  7.85398163e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  7.85398163e+00
   7.85398163e+00  1.60738706e+04]]
Eigenvalue: -92.52385546845446
Eigenstate: [-9.80995772e-02  1.92610397e-01  1.92610397e-01 -1.28389466e-01
  1.92610397e-01 -1.1475636

### Calculating Von Neumann Entanglement Entropy (VNEE)
To calculate the VNEE from the hamiltonian all you need to do is call the code below. The function takes in the constructed hamiltonian and the configuration. The configuration is a list where each index represents an atom in your system. These indices are according the conventions defined in their hamiltonian as I have described. 1 means the atom is in and 0 means the atom is out. 

Note: the calculation from the hamiltonian can take around 10-20 seconds only because the task of computing the eigen values is expensive. Does NOT currently work for systems larger than 14 atoms. Diagonalization results in memory error. If you already have the reduced density matrix you can directly calculate the VNEE using the function von_neumann_entropy_from_rdm(rdm). In this case you do not need to input your configuration as it is already implicit in your reduced density matrix. 

In [8]:
show_progress = True
state_index = 0 # ground state
configuraiton = [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
vnee = qc.von_neumann_entropy_from_hamiltonian(hamiltonian, configuraiton, state_index, show_progress)
print(vnee)


Starting: Computing Von Neumann Entropy...
Eigenstate 0 found in 12.54 seconds.                                            
Task: Computing Von Neumann Entropy | Progress: 100.00% | Elapsed: 12.85s | Remaining: 0.00s
Completed: Computing Von Neumann Entropy. Elapsed time: 12.85 seconds.
0.8440853178097764


### Getting VNEE The Long Way

If you so choose you can also use a more flexible approach to obtaining the VNEE that allows you the intermediate steps of getting the groundstate, density matrix, and reduced density matrix. The code below will provide the same answer as above but you have more oversight.

In [5]:
state = 0 # Ground state
show_progress = True #Optional Progress updates

eigen_value, eigenstate = qc.find_eigenstate(hamiltonian, state, show_progress)

density_matrix = qc.create_density_matrix(eigenstate)

configuration = [1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]
reduced_density_matrix = qc.compute_reduced_density_matrix(density_matrix, configuration)

vnee = qc.von_neumann_entropy_from_rdm(reduced_density_matrix)

print(vnee)


Starting: Finding Eigenstate...
Eigenstate 0 found in 8.50 seconds.                                             
Task: Finding Eigenstate | Progress: 100.00% | Elapsed: 8.51s | Remaining: 0.00s
Completed: Finding Eigenstate. Elapsed time: 8.51 seconds.
1.0507721786189317


### Obtaining Binary Dictionary from hamiltonian
We often work with the states and their probabilities so I've developed a function call that makes this process easier. You can choose which state you're obtaining probabilities for as well (ground, 1st, 2nd, etc...) 

Note: Obtaining eigenvalues may take up to several minutes for larger systems. Obtaining the ground state is significantly faster than obtaining any excited state. 

In [6]:
state = 0 # Get the ground state probabilities
state_prob_dict = qc.get_eigenstate_probabilities(hamiltonian, state)
qc.print_most_probable_data(state_prob_dict, 5)


Most probable 5 bit strings:
1.  Bit string: 010010010010, Probability: 0.09415953
2.  Bit string: 100001100001, Probability: 0.09415953
3.  Bit string: 100100100001, Probability: 0.07475050
4.  Bit string: 100001001001, Probability: 0.07475050
5.  Bit string: 011000010010, Probability: 0.07475050


### 1D Quantum Ising Model Chain

The code below constructs the hamiltonain of a 1D chain of spins using the Quantum Ising Model. The configuration is exactly the same as the first image for the Rydberg Chain, however we only consider nearest neighbor interactions so lattice spacing isn't needed. 

The Hamiltonian for a 1D chain of spins with a transverse magnetic field is:

\begin{equation}
H = -J \sum_{i=1}^{N-1} \sigma_x^i \sigma_x^{i+1} - h \sum_{i=1}^{N} \sigma_z^i
\end{equation}

Where J denotes the interaction strength and h denotes the transverse field strength

If periodic boundary conditions (PBC) are enabled, an additional term is included:

\begin{equation}
H_{\text{PBC}} = -J \sigma_x^N \sigma_x^1
\end{equation}


In [12]:
# Parameters for the ising system system
num_spins = 6  # Number of spins in the system
J = 1.0  # Coupling strength between neighboring spins
h = 1.0  # Strength of the transverse magnetic field
pbc = True  # Enable periodic boundary conditions

# Construct the Hamiltonian for the 1D Quantum Ising Model

hamiltonian = qc.build_ising_hamiltonian(num_spins, J, h, pbc)

print(hamiltonian.toarray())

state = 2 # Ground state
show_progress = True #Optional Progress updates

eigen_value, eigenstate = qc.find_eigenstate(hamiltonian, state, show_progress)

print(f"Eigenvalue: {eigen_value}")
print(f"Eigenstate: {eigenstate}")

[[-6.  0.  0. ...  0.  0.  0.]
 [ 0. -4. -1. ...  0.  0.  0.]
 [ 0. -1. -4. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  4. -1.  0.]
 [ 0.  0.  0. ... -1.  4.  0.]
 [ 0.  0.  0. ...  0.  0.  6.]]
Starting: Finding Eigenstate...
Eigenstate 2 found in 0.01 seconds.                                             
Task: Finding Eigenstate | Progress: 100.00% | Elapsed: 0.01s | Remaining: 0.00s
Completed: Finding Eigenstate. Elapsed time: 0.01 seconds.
Eigenvalue: -5.656854249492375
Eigenstate: [ 5.57610624e-01  3.49663332e-15 -3.19579432e-15 -3.18902860e-02
  8.53104479e-15 -2.30969883e-01 -3.18902860e-02 -2.56295907e-15
 -1.23052881e-14 -2.94750455e-01 -2.30969883e-01 -2.96191497e-15
 -3.18902860e-02 -5.50168228e-15  3.78390353e-14 -7.69899610e-02
  1.07594055e-14 -2.30969883e-01 -2.94750455e-01 -1.36061681e-14
 -2.30969883e-01 -2.45344964e-14 -1.59505655e-14 -9.56708581e-02
 -3.18902860e-02  4.13547912e-15 -1.46231361e-14 -5.83090640e-02
 -6.00231263e-15 -9.56708581e-02 -7.69899610e-02  5.71927

### Quantum Ising Ladder

Just like before we are now interested in a ladder structure. The modified hamiltonian for a 2d strucutre is:

\begin{equation}
H = -J \sum_{\langle i,j \rangle} \sigma_x^i \sigma_x^j - h \sum_{i} \sigma_z^i
\end{equation}
where:
- $ \langle i,j \rangle $ includes horizontal (in-chain), vertical (between chains), and diagonal (optional) interactions.
- $ J $ is the coupling strength.
- $ h $ is the transverse field strength.

In [8]:
# Parameters for the ising system system
num_spins = 14  # Number of spins in the system
J = 1.0  # Coupling strength between neighboring spins
h = 1.0  # Strength of the transverse magnetic field
pbc = True  # Enable periodic boundary conditions
include_diagonal = False  # Include diagonal interactions

# Construct the Hamiltonian for the 2D Quantum Ising Model

hamiltonian = qc.build_ising_hamiltonian_ladder(num_spins, J, h, pbc, include_diagonal)

print(hamiltonian.toarray())

[[-14.   0.   0. ...   0.   0.   0.]
 [  0. -12.  -1. ...   0.   0.   0.]
 [  0.  -1. -12. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...  12.  -1.   0.]
 [  0.   0.   0. ...  -1.  12.   0.]
 [  0.   0.   0. ...   0.   0.  14.]]
